In [7]:
#| warning: false
library(dplyr)
library(dtplyr)
library(data.table)
library(lubridate)
options(warn=-1)

In [25]:
df = fread("compustat_daily_2010_2025.csv")
glimpse(df)

Rows: 783,992
Columns: 20
$ tic      <chr> "AIR", "AIR", "AIR", "AIR", "AIR", "AIR", "AIR", "AIR", "AIR"…
$ datadate <chr> "01/04/2010", "01/05/2010", "01/06/2010", "01/07/2010", "01/0…
$ conm     <chr> "AAR CORP", "AAR CORP", "AAR CORP", "AAR CORP", "AAR CORP", "…
$ exchg    <int> 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1…
$ add1     <chr> "One AAR Place, 1100 North Wood Dale Road", "One AAR Place, 1…
$ addzip   <chr> "60191", "60191", "60191", "60191", "60191", "60191", "60191"…
$ busdesc  <chr> "AAR Corp. provides products and services to commercial aviat…
$ city     <chr> "Wood Dale", "Wood Dale", "Wood Dale", "Wood Dale", "Wood Dal…
$ conml    <chr> "AAR Corp", "AAR Corp", "AAR Corp", "AAR Corp", "AAR Corp", "…
$ fax      <chr> "630 227 2039", "630 227 2039", "630 227 2039", "630 227 2039…
$ loc      <chr> "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA", "USA"…
$ phone    <chr> "630 227 2000", "630 227 2000", "630 227 2000", "630 227 2000…
$ sic      <in